In [ ]:

import pandas as pd
import matplotlib.pyplot as plt

In [7]:
# makes the plots readable

plt.rcParams.update({
   'font.size': 12,
   'axes.labelsize': 12,
   'axes.titlesize': 14,
   'xtick.labelsize': 11,
   'ytick.labelsize': 11,
   'legend.fontsize': 11
})

In [ ]:
df = pd.read_csv('../data/processed/merged_data.csv', index_col=0, parse_dates=True)

df = df.replace(['empty', 'n/e'], pd.NA)
df = df.apply(pd.to_numeric, errors='coerce')

In [ ]:
# Calculate daily averages for prices and auction data
weekly_df = df.resample('W').mean()

# Calculate price differences 
weekly_df['DE-LU_minus_CH'] = weekly_df['DE-LU'] - weekly_df['CH']
weekly_df['DE-AT-LU_minus_CH'] = weekly_df['DE-AT-LU'] - weekly_df['CH']

plt.figure(figsize=(12, 6))

# Plot DE-LU minus CH price difference
plt.plot(
   weekly_df.index,
   weekly_df['DE-LU_minus_CH'],
   label='Day-Ahead Price Difference',
   color='blue'
)

# Fill between price difference and auction price
plt.fill_between(
   weekly_df.index,
   weekly_df['DE-LU_minus_CH'],
   weekly_df['DE-LU_minus_CH'] + weekly_df['de-ch_auctionPrice'],
   color='skyblue',
   alpha=0.3,
   label='Auction Price'
)

plt.ylabel('Daily Average Price [EUR/MWh]')
plt.title('Daily Average Day-ahead Price Difference (DE-LU minus CH) and Auction Price')
plt.legend()
plt.grid(True)
plt.show()

# Same plot for DE-AT-LU
plt.figure(figsize=(12, 6))
plt.plot(
   weekly_df.index,
   weekly_df['DE-AT-LU_minus_CH'],
   label='Day-Ahead Price Difference',
   color='blue'
)

plt.fill_between(
   weekly_df.index,
   weekly_df['DE-AT-LU_minus_CH'],
   weekly_df['DE-AT-LU_minus_CH'] + weekly_df['de-ch_auctionPrice'],
   color='skyblue',
   alpha=0.3,
   label='Auction Price'
)

plt.ylabel('Daily Average Price [EUR/MWh]')
plt.title('Daily Average Day-ahead Price Difference (DE-AT-LU minus CH) and Auction Price')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Calculate hourly price difference
df['CH_minus_DE'] = df['CH'] - df['DE-LU']

# Analyze error distribution
error = df['CH_minus_DE'] - df['de-ch_auctionPrice']
plt.figure(figsize=(12, 6))
plt.hist(error.dropna(), bins=100, alpha=0.75, color='blue', edgecolor='black')
plt.xlabel('Error [EUR/MWh]')
plt.ylabel('Frequency')
plt.title('Error Distribution (CH - DE minus Auction Price)')
plt.grid(True)
plt.show()

# Analyze variance at different times of the day
df['hour'] = df.index.hour
hourly_variance = df.groupby('hour')['CH_minus_DE'].var()

plt.figure(figsize=(12, 6))
plt.plot(hourly_variance.index, hourly_variance, marker='o', linestyle='-', color='blue')
plt.xlabel('Hour of the Day')
plt.ylabel('Variance [EUR^2/MWh^2]')
plt.title('Variance of Hourly Price Difference (CH - DE) by Hour of the Day')
plt.grid(True)
plt.show()